In [6]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np

# ------------------------------
# 1️⃣ Загружаем границы города из локального файла
# ------------------------------
geojson_file = "kazan_boundary_local.geojson"  # <- твой файл
print(f"[*] Загружаем границы из {geojson_file}...")
city = gpd.read_file('kazan_map.geojson')
boundary_shape = city.geometry.iloc[0]
print("[+] Границы города загружены")

# ------------------------------
# 2️⃣ Генерация сетки (квадраты 500 м)
# ------------------------------
print("[*] Генерируем сетку 500x500 м...")

# Получаем bounding box города
minx, miny, maxx, maxy = boundary_shape.bounds

# Шаг в градусах (~0.0045° ~ 500 м)
delta = 0.0045  

latitudes = np.arange(miny, maxy, delta)
longitudes = np.arange(minx, maxx, delta)

created_segments = []
count = 0

for lat in latitudes:
    for lon in longitudes:
        polygon = Polygon([
            (lon, lat),
            (lon + delta, lat),
            (lon + delta, lat + delta),
            (lon, lat + delta),
            (lon, lat)
        ])
        if boundary_shape.intersects(polygon):
            count += 1
            created_segments.append({
                "name": f"Kazan segment {count}",
                "polygon": polygon
            })

print(f"[+] Сетка готова: {count} квадратов внутри города")

# ------------------------------
# 3️⃣ Визуализация на карте
# ------------------------------
print("[*] Рисуем карту...")

# Центр карты — центр границы
centroid = boundary_shape.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12)

# Границы города
folium.GeoJson(city.geometry.__geo_interface__,
               name="Kazan boundary",
               style_function=lambda x: {"color": "red", "weight": 3, "fill": False}).add_to(m)

# Квадраты
for seg in created_segments:
    coords = [(lat, lon) for lon, lat in seg["polygon"].exterior.coords]
    folium.Polygon(
        locations=coords,
        color='blue',
        weight=1,
        fill=True,
        fill_opacity=0.3,
        popup=seg["name"]
    ).add_to(m)

m.save("kazan_grid_local.html")
print("[✅] Карта сохранена как kazan_grid_local.html — открывай в браузере!")


[*] Загружаем границы из kazan_boundary_local.geojson...
[+] Границы города загружены
[*] Генерируем сетку 500x500 м...
[+] Сетка готова: 810 квадратов внутри города
[*] Рисуем карту...
[✅] Карта сохранена как kazan_grid_local.html — открывай в браузере!
